---
# MLlib Spark

## Aprendizado de Máquina no Apache Spark

Alunos:


*   **Alisson Nunes** (725862)
*   **Lucas Mathaeus** (726561)
*   **William Eugênio** (726601)

Bacharelado em Ciência da Computação (BCC)<br>
Departamento de Computação (DC)<br>
Universidade Federal de São Carlos (UFSCar)


#  (1) Obtenção dos Dados

In [ ]:
#instalando o módulo wget
#%%capture
%pip install -q wget
%mkdir data

In [ ]:
#baixando os dados das tabelas de dimensão do data mart Exame
import wget

url = "https://raw.githubusercontent.com/cristinaaguiar/dataMartExame/main/idade.csv"
wget.download(url, "data/idade.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/dataMartExame/main/data.csv"
wget.download(url, "data/data.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/dataMartExame/main/descreveExame.csv"
wget.download(url, "data/descreveExame.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/dataMartExame/main/hospital.csv"
wget.download(url, "data/hospital.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/dataMartExame/main/exame.csv"
wget.download(url, "data/exame.csv")

In [13]:
#instalando os pacotes para o uso de Spark em Python
# %%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz
!rm spark-3.0.0-bin-hadoop2.7.tgz
%pip install -q findspark
%pip install -q pyspark==3.0.0

O sistema n�o pode encontrar o caminho especificado.
UsageError: Line magic function `%wget` not found.


In [ ]:
#configurando as variaveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

In [ ]:
#instalando o pacote findspark
import findspark
findspark.init()

In [ ]:
#definindo uma sessão Spark para usar o pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark-notebook").master("local[*]").getOrCreate()

(remover ele chama o banco assim: 

#criando e exibindo o DataFrame em Spark para a tabela de dimensão idade
idade = spark.read.csv(path="data/idade.csv", header=True, sep=",")
idade.show()
)

# (2) Instalação e Configuração

In [2]:
! pip install pyspark